In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm import tqdm
from deepctr.models import DeepFM
from deepctr.layers import custom_objects
from tensorflow.data import Dataset as tfds

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.4.4
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


In [2]:
from os.path import isfile, join
basedir = '/home/lms/ms/poc'
rawdata_path = join(basedir, 'dataset')
procdata_path = join(basedir, 'deepfm', 'dataset', 'preprocess', 'rating')
procdata_path_5 = join(basedir, 'deepfm', 'dataset', 'preprocess', '5_1')
model_path = join(basedir, 'deepfm', 'model', 'model_5_f')
result_path = join(basedir, 'deepfm', 'result', 'result_5_f')

result_path_5_no_f = join(basedir, 'deepfm', 'result', 'result_5_no_f') # path 설정 필요
result_path_r_no_f = join(basedir, 'deepfm', 'result', 'result_r_no_f') # path 설정 필요
result_path_5_f = join(basedir, 'deepfm', 'result', 'result_5_f') # path 설정 필요
result_path_r_f = join(basedir, 'deepfm', 'result', 'result_r_f') # path 설정 필요

model_path_5_no_f= join(basedir, 'deepfm', 'model', 'model_5_no_f')
model_path_r_no_f= join(basedir, 'deepfm', 'model', 'model_r_no_f')
model_path_5_f= join(basedir, 'deepfm', 'model', 'model_5_f')
model_path_r_f= join(basedir, 'deepfm', 'model', 'model_r_f')

rawdata_path = rawdata_path
procdata_path = procdata_path
model_path = model_path
batch_size = 50
topk = 25

In [3]:
#load test data
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
data = pd.read_pickle(os.path.join(procdata_path, 'hist.pkl'))
train, test = train_test_split(data, test_size=0.2, random_state=2020)
test.head()

,sa_id_CODE,category_id_CODE,rating_r,rating_5,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
10838611,503022,1188,4.50000,5,0.85,0.0,0.07,0.08,0,2,0.926502,0.201008
15979174,652914,11948,4.50000,5,0.87,0.0,0.00,0.13,0,2,0.947011,0.192812
4541125,502835,166,5.00000,5,0.98,0.0,0.01,0.01,3,0,0.999991,0.155527
14053222,352951,3104,4.50000,5,0.94,0.0,0.00,0.06,0,3,0.989897,0.170110
8806112,643387,669,2.61375,5,1.00,0.0,0.00,0.00,1,3,0.883084,0.240833


In [5]:
test

,sa_id_CODE,category_id_CODE,rating_r,rating_5,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
10838611,503022,1188,4.50000,5,0.85,0.0,0.07,0.08,0,2,0.926502,0.201008
15979174,652914,11948,4.50000,5,0.87,0.0,0.00,0.13,0,2,0.947011,0.192812
4541125,502835,166,5.00000,5,0.98,0.0,0.01,0.01,3,0,0.999991,0.155527
14053222,352951,3104,4.50000,5,0.94,0.0,0.00,0.06,0,3,0.989897,0.170110
8806112,643387,669,2.61375,5,1.00,0.0,0.00,0.00,1,3,0.883084,0.240833
...,...,...,...,...,...,...,...,...,...,...,...,...
10724696,1302475,1138,4.55350,5,0.00,0.0,1.00,0.00,0,3,0.958395,0.250645
4988436,41611,192,5.00000,5,1.00,0.0,0.00,0.00,0,1,0.999994,0.145908
12062498,1326121,1779,4.99275,5,1.00,0.0,0.00,0.00,0,3,0.926293,0.231444
14768949,912281,4051,4.87825,5,0.22,0.0,0.11,0.67,0,3,0.989781,0.254377


In [ ]:
user_file='user.pkl'
item_file='item.pkl'

users = pd.read_pickle(os.path.join(procdata_path, user_file))
items = pd.read_pickle(os.path.join(procdata_path, item_file))

In [ ]:
item_columns = ['category_id','gubun','rep_album_id','category_name']
category_data = pd.read_pickle(os.path.join(rawdata_path, 'item.pkl'))
category_data = category_data.astype({'category_id':'str', 'category_name':'str', 'album_id':'str'})
category_data = category_data.drop_duplicates(subset=item_columns)

n_users = len(users)
n_items = len(items)

category_data = items.merge(category_data, on='category_id')

In [ ]:
unique_user = users[users.columns[1:]]
unique_item = items[items.columns[1:]]

In [6]:
model = tf.keras.models.load_model(os.path.join(model_path_5_f, 'bestmodel'), 
                                   custom_objects=custom_objects)

In [7]:
test = test.sort_values(by=['sa_id_CODE'])
test

,sa_id_CODE,category_id_CODE,rating_r,rating_5,seg_1,seg_2,seg_3,seg_4,pr_info_CODE,price_CODE,release_date,run_time_ss
203378,1,1,5.00000,5,0.97,0.0,0.03,0.00,0,0,0.990313,0.131286
318085,1,4,5.00000,5,0.97,0.0,0.03,0.00,2,0,0.979802,0.163261
548474,2,12,4.50000,5,0.97,0.0,0.03,0.00,0,0,0.999996,0.186040
1957412,3,55,4.50000,5,0.98,0.0,0.01,0.01,0,0,0.990521,0.372504
1949034,3,54,3.54225,5,0.98,0.0,0.01,0.01,0,0,0.999995,0.190927
...,...,...,...,...,...,...,...,...,...,...,...,...
16043239,1973436,16732,4.25950,5,0.00,0.0,1.00,0.00,0,3,0.979699,0.217938
16043632,1973441,16912,4.50000,5,0.00,0.0,0.00,1.00,3,3,0.820103,0.223094
16043719,1973445,16951,4.98875,5,0.00,1.0,0.00,0.00,0,14,0.925741,0.105352
16043794,1973447,16994,4.74375,5,1.00,0.0,0.00,0.00,1,3,0.915641,0.203317


In [8]:
test_user = test['sa_id_CODE'].unique()
test_user

array([      1,       2,       3, ..., 1973445, 1973447, 1973449])

In [9]:
len(test_user)

1137481